#Load data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [4]:
# intents
!mkdir -p dialogue_018/intents/NONE
!mkdir -p dialogue_018/intents/SearchOnewayFlight
!mkdir -p dialogue_018/intents/ReserveOnewayFlight
!mkdir -p dialogue_018/intents/SearchRoundtripFlights
!mkdir -p dialogue_018/intents/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [7]:
import json
import os

if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/intents/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()

                # Dados para o extrator de entidades
                slots_dict = current_frame['frames'][0]['state']['slot_values']
                all_values = [value for values in slots_dict.values() for value in values]

                for each_value in all_values:
                  base_path = "dialogue_018/" + each_value + "/"
                  file_path_2 = "dialogue_018/" + each_value + "/" +  str(i) + "_" + str(j) + ".txt"
                  # Check if the directory exists
                  if not os.path.exists(base_path):
                      # If not, create the directory
                      os.makedirs(base_path)
                  f = open(file_path_2, "w+")
                  f.write(utterance)
                  f.close()

Armazenar numa estrutura para treinamento no Keras

### Dados para o classificador

In [8]:
import tensorflow as tf

seed = 42

raw_train_ds= tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018/intents',
    labels='inferred',
    validation_split=0.2,
    subset='training',
    seed=seed,
    shuffle=False)

raw_validation_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018/intents',
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    shuffle=False)

Found 1052 files belonging to 5 classes.
Using 842 files for training.
Found 1052 files belonging to 5 classes.
Using 210 files for validation.


In [9]:
raw_train_ds.class_names

['NONE',
 'ReserveOnewayFlight',
 'ReserveRoundtripFlights',
 'SearchOnewayFlight',
 'SearchRoundtripFlights']

In [ ]:
for text_batch, label_batch in raw_train_ds:
  print("Utterance: ", text_batch)
  print("Label: ", label_batch)

In [ ]:
print(raw_train_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [26]:
import tensorflow as tf

seed = 42

raw_extractor_train_ds= tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018/',
    labels='inferred',
    validation_split=0.2,
    subset='training',
    seed=seed,
    shuffle=False)

raw_extractor_validation_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018/',
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    shuffle=False)

Found 8068 files belonging to 296 classes.
Using 6455 files for training.
Found 8068 files belonging to 296 classes.
Using 1613 files for validation.


In [ ]:
raw_extractor_train_ds.class_names

### Dados para o extrator

Acho que o Tensor acima é dividido em batches e acredito que isso adiciona uma complexidade na hora de tokenizar, pois teríamos que fazer para cada um dos tensores. Testei resolver fazer uma concatenação de todos os tensores abaixo:

Tensores para o classificador de intenções

In [10]:
all_text_tensor = tf.concat([text_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_text_tensor

In [11]:
all_labels = tf.concat([label_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_labels

In [12]:
utterances_validation_tensor = tf.concat([text_batch for text_batch, label_batch in raw_validation_ds], axis=0)

In [13]:
validation_labels = tf.concat([label_batch for text_batch, label_batch in raw_validation_ds], axis=0)

Tensores para o extrator de entidades

In [28]:
extractor_texts_tensor = tf.concat([text_batch for text_batch, label_batch in raw_extractor_train_ds], axis=0)
extractor_texts_tensor

<tf.Tensor: shape=(6455,), dtype=string, numpy=
array([b'Yep that sounds good. What airport will I be flying into and what time will it arrive?',
       b'Thanks for your help.', b'No, that will be all.', ...,
       b'That is great.', b'Thanks. That is all.',
       b'That is good. Can you buy me tickets?'], dtype=object)>

In [30]:
extractor_labels_tensor = tf.concat([label_batch for text_batch, label_batch in raw_extractor_train_ds], axis=0)
extractor_labels_tensor

<tf.Tensor: shape=(6455,), dtype=int32, numpy=array([  0,   0,   0, ..., 251, 251, 251], dtype=int32)>

In [32]:
extractor_validation_tensor = tf.concat([text_batch for text_batch, label_batch in raw_extractor_validation_ds], axis=0)

In [34]:
extractor_validation_labels = tf.concat([label_batch for text_batch, label_batch in raw_extractor_validation_ds], axis=0)

# Pre processamento de dados
Fazer uma bag-of-words

In [14]:
from keras.preprocessing.text import Tokenizer
import json
import numpy
from keras.preprocessing import text, sequence

### Train subset

In [ ]:
t = Tokenizer()
texts = all_text_tensor.numpy().astype(str)
t.fit_on_texts(texts)

#print(t.document_count)
print(t.word_index)
#print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(texts, mode='count')
print(encoded_docs)

In [16]:
seq_token = t.texts_to_sequences(texts)
padded_sequences = sequence.pad_sequences(seq_token)

In [17]:
padded_sequences.shape

(842, 30)

In [18]:
all_labels.shape

TensorShape([842])

### Validation subset

In [19]:
t2 = Tokenizer()
validation_texts = utterances_validation_tensor.numpy().astype(str)
t2.fit_on_texts(validation_texts)
validation_seq_token = t2.texts_to_sequences(validation_texts)
padded_validation = sequence.pad_sequences(validation_seq_token)
#padded_validation = sequence.pad_sequences(validation_seq_token, maxlen=30)

In [20]:
padded_validation.shape

(210, 28)

In [21]:
validation_labels.shape

TensorShape([210])

# Classificação de intenções (Parte 1)
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM </br>
https://keras.io/examples/nlp/text_classification_with_transformer/ </br>

## LSTM

In [22]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM
from keras.layers import Embedding
import keras.backend as K

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=842, output_dim=100))
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(5,activation='softmax'))
model_lstm.compile(loss=['sparse_categorical_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         84200     
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 126765 (495.18 KB)
Trainable params: 126765 (495.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
history_lstm = model_lstm.fit(padded_sequences, all_labels, epochs=10)

Epoch 1/10
27/27 [==============================] - 4s 22ms/step - loss: 1.3942 - accuracy: 0.4893
Epoch 2/10
27/27 [==============================] - 1s 31ms/step - loss: 1.0900 - accuracy: 0.6081
Epoch 3/10
27/27 [==============================] - 1s 34ms/step - loss: 0.8570 - accuracy: 0.7114
Epoch 4/10
27/27 [==============================] - 1s 35ms/step - loss: 0.6896 - accuracy: 0.7435
Epoch 5/10
27/27 [==============================] - 1s 30ms/step - loss: 0.6102 - accuracy: 0.7969
Epoch 6/10
27/27 [==============================] - 1s 22ms/step - loss: 0.5513 - accuracy: 0.8124
Epoch 7/10
27/27 [==============================] - 1s 22ms/step - loss: 0.4720 - accuracy: 0.8325
Epoch 8/10
27/27 [==============================] - 1s 21ms/step - loss: 0.4157 - accuracy: 0.8610
Epoch 9/10
27/27 [==============================] - 1s 22ms/step - loss: 0.3640 - accuracy: 0.8765
Epoch 10/10
27/27 [==============================] - 1s 22ms/step - loss: 0.3428 - accuracy: 0.8848


In [23]:
history_with_validation_lstm = model_lstm.fit(padded_sequences, all_labels, validation_data=(padded_validation, validation_labels), epochs=10)

Epoch 1/10
27/27 [==============================] - 4s 48ms/step - loss: 1.3926 - accuracy: 0.4287 - val_loss: 0.8451 - val_accuracy: 1.0000
Epoch 2/10
27/27 [==============================] - 1s 23ms/step - loss: 1.1208 - accuracy: 0.6045 - val_loss: 0.8833 - val_accuracy: 0.6857
Epoch 3/10
27/27 [==============================] - 1s 22ms/step - loss: 0.8520 - accuracy: 0.6995 - val_loss: 0.8679 - val_accuracy: 0.6286
Epoch 4/10
27/27 [==============================] - 1s 22ms/step - loss: 0.6662 - accuracy: 0.7565 - val_loss: 1.5056 - val_accuracy: 0.4905
Epoch 5/10
27/27 [==============================] - 1s 23ms/step - loss: 0.5582 - accuracy: 0.8207 - val_loss: 1.3898 - val_accuracy: 0.5619
Epoch 6/10
27/27 [==============================] - 1s 23ms/step - loss: 0.4805 - accuracy: 0.8504 - val_loss: 1.8051 - val_accuracy: 0.4810
Epoch 7/10
27/27 [==============================] - 1s 22ms/step - loss: 0.4161 - accuracy: 0.8563 - val_loss: 2.0225 - val_accuracy: 0.4286
Epoch 8/10
27

## SVM

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

with open("dialogues_018.json", 'r') as file:
    data_018 = json.load(file)

texts = []
labels = []

for i in range(len(data_018)):
    frames_amount = len(data_018[i]['turns'])
    for j in range(frames_amount):
        current_frame = data_018[i]['turns'][j]
        speaker = current_frame['speaker']
        if speaker == 'USER':
            intent = current_frame['frames'][0]['state']['active_intent']
            utterance = current_frame['utterance']
            texts.append(utterance)
            labels.append(intent)

texts_train, texts_validation, labels_train, labels_validation = train_test_split(texts, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts_train)
X_validation = vectorizer.transform(texts_validation)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, labels_train)

predictions = svm_model.predict(X_validation)

accuracy = accuracy_score(labels_validation, predictions)
print(f'Acurácia do modelo SVM: {accuracy}')


Acurácia do modelo SVM: 0.8104265402843602


In [ ]:
# Exemplo de diálogo
example_dialogue = [
    {"speaker": "USER", "utterance": "I would like a short trip to New York"}
    # Adicione mais turnos do diálogo conforme necessário
]

# Pré-processamento do exemplo de diálogo
example_text = example_dialogue[0]['utterance']
example_vector = vectorizer.transform([example_text])

# Previsão do modelo SVM
predicted_intent = svm_model.predict(example_vector)

# Resultados
print(f"Texto do Usuário: {example_text}")
print(f"Intenção Prevista: {predicted_intent[0]}")


Texto do Usuário: I would like a short trip to New York
Intenção Prevista: SearchRoundtripFlights


## CNN

In [ ]:
import pandas as pd
import numpy as np
import time

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=842, output_dim=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Alterado para categorical_crossentropy
              metrics=['accuracy'])

print(model.summary())

# Convertendo os rótulos para codificação one-hot
from tensorflow.keras.utils import to_categorical
labels_one_hot = to_categorical(all_labels, num_classes=5)

# Fit do modelo
history = model.fit(padded_sequences, labels_one_hot, epochs=10, validation_data=(padded_validation, to_categorical(validation_labels, num_classes=5)), batch_size=5)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 100)         84200     
                                                                 
 conv1d_8 (Conv1D)           (None, None, 128)         64128     
                                                                 
 global_max_pooling1d_8 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 156909 (612.93 KB)
Trainable params: 156909 (612.93 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
# Supondo que você tenha um novo texto a ser classificado
new_text = "i need to buy a one way flight"

new_sequence = t.texts_to_sequences([new_text])
padded_new_sequence = pad_sequences(new_sequence, maxlen=10, padding='post')

predictions = model.predict(padded_new_sequence)

predicted_class = predictions.argmax(axis=-1)

print(f'O texto foi classificado como classe: {predicted_class[0]}')


1/1 [==============================] - 0s 115ms/step
O texto foi classificado como classe: 1


## Teste de transformer com BERT

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.layers import Dense

In [ ]:
# Load BERT tokenizer and model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Modify the model's classifier layer
classifier_layer = Dense(5, activation='softmax',  name='classifier')
bert_model.layers[-1]= classifier_layer

In [ ]:
import keras
opt = keras.optimizers.Adam(learning_rate=2e-5)
bert_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bert_history = bert_model.fit(
    padded_sequences,
    all_labels,
    epochs=3
)

Epoch 1/3
27/27 [==============================] - 328s 11s/step - loss: 3.1591 - accuracy: 0.3753
Epoch 2/3
27/27 [==============================] - 279s 10s/step - loss: 1.9506 - accuracy: 0.3717
Epoch 3/3
27/27 [==============================] - 283s 10s/step - loss: 1.4578 - accuracy: 0.4204


# Extrator de entidades (Parte 2)
Podemos definir algumas entidades:
- Flight number
- Airline
- Departure time
- Arrival time
- Flight duration
- Amount of passengers

### LSTM (Extrator)

In [38]:
t3 = Tokenizer()
texts = extractor_texts_tensor.numpy().astype(str)
t3.fit_on_texts(texts)
seq_token = t3.texts_to_sequences(texts)
padded_sequences_extractor_lstm = sequence.pad_sequences(seq_token)

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

extractor_lstm = Sequential()
extractor_lstm.add(Embedding(input_dim=6455, output_dim=50))
extractor_lstm.add(LSTM(units=32))
# The number of units in Dense layer should be equal to the number of entity labels + 1
extractor_lstm.add(Dense(296, activation='softmax'))
extractor_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
extractor_lstm.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 50)          322750    
                                                                 
 lstm_7 (LSTM)               (None, 32)                10624     
                                                                 
 dense_7 (Dense)             (None, 296)               9768      
                                                                 
Total params: 343142 (1.31 MB)
Trainable params: 343142 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
history_extractor_lstm = extractor_lstm.fit(padded_sequences_extractor_lstm, extractor_labels_tensor, epochs=10)


Epoch 1/10
202/202 [==============================] - 6s 19ms/step - loss: 5.0062 - accuracy: 0.0542
Epoch 2/10
202/202 [==============================] - 5s 25ms/step - loss: 4.7712 - accuracy: 0.0634
Epoch 3/10
202/202 [==============================] - 4s 19ms/step - loss: 4.7628 - accuracy: 0.0626
Epoch 4/10
202/202 [==============================] - 4s 19ms/step - loss: 4.7561 - accuracy: 0.0634
Epoch 5/10
202/202 [==============================] - 5s 25ms/step - loss: 4.7491 - accuracy: 0.0634
Epoch 6/10
202/202 [==============================] - 4s 19ms/step - loss: 4.7428 - accuracy: 0.0652
Epoch 7/10
202/202 [==============================] - 4s 19ms/step - loss: 4.7332 - accuracy: 0.0660
Epoch 8/10
202/202 [==============================] - 5s 24ms/step - loss: 4.7244 - accuracy: 0.0679
Epoch 9/10
202/202 [==============================] - 4s 20ms/step - loss: 4.7152 - accuracy: 0.0669
Epoch 10/10
202/202 [==============================] - 4s 19ms/step - loss: 4.7042 - accura

### Transformer (Extrator)

# Conversação

In [41]:
def preprocess_input(user_input):
  t = Tokenizer()
  t.fit_on_texts(user_input)
  seq_token = t.texts_to_sequences([user_input])
  padded_sequences = sequence.pad_sequences(seq_token)
  return padded_sequences

In [42]:
import numpy as np
def classify_intent(preprocessed_input):
  predictions = model_lstm.predict(np.array(preprocessed_input))

  predicted_intent_index = np.argmax(predictions)
  confidence = predictions[0][predicted_intent_index]

  intents = ["ReserveOnewayFlight", "SearchOnewayFlight", "SearchRoundtripFlights", "ReserveRoundtripFlights", "NONE"]

  predicted_intent = intents[predicted_intent_index]

  return predicted_intent, confidence

In [44]:
def extract_entities(preprocessed_input):
  predictions = extractor_lstm.predict(preprocessed_input)
  possible_entities = raw_extractor_train_ds.class_names
  entity_labels = [possible_entities[i] for i, pred in enumerate(predictions[0]) if pred > 0.5]

  return entity_labels

In [ ]:
def generate_response(combined_info):
  pass

In [45]:
while True:
  user_input = input("User: ")

  if user_input == "Bye":
    break

  preprocessed_input = preprocess_input(user_input)
  #print(preprocessed_input.shape)

  # Intent Classification
  intent, confidence = classify_intent(preprocessed_input)

  # Entity Extraction
  entities = extract_entities(preprocessed_input)

  # Combine Information
  combined_info = {'intent': intent, 'confidence': confidence, 'entities': entities}

  # Dialogue Management and Response Generation
  #response = generate_response(combined_info)

  print(f"Intent: {intent}, Confidence: {confidence}")
  print(f"Entities: {entities}")
  #print(f"Bot: {response}\n")

User: I would like a one way flight to Atlanta
1/1 [==============================] - 0s 436ms/step
Intent: SearchRoundtripFlights, Confidence: 0.4188711643218994
Entities: []
User: Bye
